In [2]:
from datetime import datetime, timedelta

now = datetime.now()
print(now.hour)

11


In [1]:
import random
# print(random.choices(range(0,5), k=13))
for i in range(2):
    print(i)
    

0
1


In [6]:
foo = [3, 3, 4]

for i in range(len(foo)):
    if 3 == foo[i]:
        print("3!")


3!
3!


In [12]:
random.sample(range(0, 6), 5)

[2, 4, 1, 3, 5]

In [15]:
for i in range(0, 6):
    print(i)

0
1
2
3
4
5
